In [1]:
# import
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV, LinearRegression

%matplotlib inline
import gc

In [2]:
df = pd.read_csv('C:/Users/y/Desktop/424brooklyn/data/lintong/COS424_final/workingdata.csv')
# df = pd.read_csv('/home/yingyig/data/workingdata.csv')
df.head()

,land_sqft,gross_sqft,sale_price,year_of_sale,easements,lotarea,bldgarea,comarea,resarea,officearea,...,edesignum110,edesignum111,edesignum112,edesignum113,edesignum114,edesignum115,edesignum116,firm07_fla2,pfirm15_fl2,year_last_alter
0,143711.0,NaN,NaN,2003,0,143711.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1,323.0,NaN,NaN,2004,0,323.0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,NaN
2,2000.0,NaN,NaN,2005,0,2000.0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,NaN
3,23000.0,NaN,NaN,2005,1,23000.0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,NaN
4,245065.0,NaN,NaN,2005,0,245065.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


# pre-processing

In [3]:
# get rid of missing sale_price and outliers
sale_prc_threshold = 10000  # to be determined
df_null_filtered = df[(~df.sale_price.isnull()) & (df.sale_price >= sale_prc_threshold)].reset_index(drop=True)
df_sale_missing = df[df.sale_price.isnull()]
del df

# split training and test sets before pre-processing
frac = 0.25
ntest = int(len(df_null_filtered) * frac)
test_indices = [int(np.floor(len(df_null_filtered) * np.random.rand())) for i in range(ntest)] 
train_indices = list(set(df_null_filtered.index) - set(test_indices))
df_train = df_null_filtered.iloc[train_indices, :]
df_test = df_null_filtered.iloc[test_indices, :]
del df_null_filtered

# impute with column mean, will replace with other methods later.
# set aside sale_prc missing dataset
df_train_imputed = df_train.fillna(df_train.mean())
df_test_imputed = df_test.fillna(df_test.mean())
df_sale_missing_imputed = df_sale_missing.fillna(df_sale_missing.mean()).drop(columns=['sale_price'])
del df_train
del df_test
del df_sale_missing

# separate X and y, and take log scale of y
X_train = df_train_imputed.loc[:, df_train_imputed.columns != 'sale_price']
y_train = np.log10(df_train_imputed.sale_price)
X_test = df_test_imputed.loc[:, df_test_imputed.columns != 'sale_price']
y_test = np.log10(df_test_imputed.sale_price)
del df_train_imputed
del df_test_imputed

gc.collect()

36169

### modeling

In [4]:
# prepare models
models = []
models.append(('LR', LinearRegression()))
#models.append(('Lasso', Lasso()))
#models.append(('Ridge', Ridge()))
#models.append(('EN', ElasticNet()))
#models.append(('LassoCV', LassoCV()))
#models.append(('RidgeCV', RidgeCV()))
#models.append(('ENCV', ElasticNetCV()))

# evaluate each model in turn
names = []
for name, model in models:
    model.fit(X_train, y_train)
    # adjusted r^2 on test set
    print("\n%s --- %f" % (name, model.score(X_test, y_test)))  
    # if applied on missing sale_prc items
    y_missing_pred = model.predict(df_sale_missing_imputed)


LR --- 0.494917
